In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import zipfile
import os

In [17]:
data = pd.read_csv("C:/연구인턴십/filtered_travel_data.csv")

In [3]:
data

,TRAVEL_ID,PAYMENT_AMT_WON,PAYMENT_AMT_WON_lodge,PAYMENT_AMT_WON_mvmn,INCOME,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_COMPANIONS_NUM,TRAVEL_PURPOSE,MVMN_NM,TRAVEL_DAY,START_DAY_OF_WEEK,END_DAY_OF_WEEK,WEEKEND_INCLUDED,SEASON,Total_Cost_x,Total_Cost_y,Total_Expense
0,a_a000007,12000.0,1400000.0,50000.0,4.0,서울특별시,인천,5.0,10;23;24;26;7;,자가용,4.0,Friday,Monday,1,여름,NaN,41492400.0,41492400.0
1,a_a000007,12000.0,1400000.0,4000.0,4.0,서울특별시,인천,5.0,10;23;24;26;7;,자가용,4.0,Friday,Monday,1,여름,NaN,41492400.0,41492400.0
2,a_a000007,12000.0,1400000.0,240000.0,4.0,서울특별시,인천,5.0,10;23;24;26;7;,자가용,4.0,Friday,Monday,1,여름,NaN,41492400.0,41492400.0
3,a_a000007,12000.0,1400000.0,50000.0,4.0,서울특별시,인천,5.0,10;23;24;26;7;,자가용,4.0,Friday,Monday,1,여름,NaN,41492400.0,41492400.0
4,a_a000007,12000.0,1400000.0,4000.0,4.0,서울특별시,인천,5.0,10;23;24;26;7;,자가용,4.0,Friday,Monday,1,여름,NaN,41492400.0,41492400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621666,NaN,NaN,NaN,NaN,5.0,제주특별자치도,제주,0.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
621667,NaN,NaN,NaN,NaN,4.0,제주특별자치도,제주,0.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
621668,NaN,NaN,NaN,NaN,4.0,제주특별자치도,제주,0.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
621669,NaN,NaN,NaN,NaN,2.0,제주특별자치도,제주,0.0,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 621671 entries, 0 to 621670
Data columns (total 42 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   TRAVEL_ID                  621671 non-null  object 
 1   VISIT_AREA_ID              617798 non-null  float64
 2   ACTIVITY_TYPE_CD           617798 non-null  float64
 3   PAYMENT_NUM                617798 non-null  float64
 4   ROAD_NM_CD                 498216 non-null  float64
 5   LOTNO_CD                   498216 non-null  float64
 6   PAYMENT_AMT_WON            617558 non-null  float64
 7   PAYMENT_ETC                378109 non-null  object 
 8   SGG_CD                     94692 non-null   float64
 9   REL_CD                     510947 non-null  float64
 10  COMPANION_SITUATION        510947 non-null  float64
 11  LODGING_TYPE_CD            519140 non-null  float64
 12  RSVT_YN                    491020 non-null  object 
 13  PAYMENT_NUM_lodge          51

1. 주말 변수 추가하기

In [33]:
## 주말 변수 추가하기


# 날짜 형식으로 변환
data['TRAVEL_START_YMD'] = pd.to_datetime(data['TRAVEL_START_YMD'])
data['TRAVEL_END_YMD'] = pd.to_datetime(data['TRAVEL_END_YMD'])

# 시작일과 종료일의 요일 정보 추가
data['START_DAY_OF_WEEK'] = data['TRAVEL_START_YMD'].dt.strftime('%A')
data['END_DAY_OF_WEEK'] = data['TRAVEL_END_YMD'].dt.strftime('%A')

# 주말 포함 여부 확인 함수 (NaT 값 처리 추가)
def check_weekend_included(start_date, end_date):
    if pd.isna(start_date) or pd.isna(end_date):
        return -1  # NaT 값이 있는 경우 -1을 반환
    date_range = pd.date_range(start=start_date, end=end_date)
    return 1 if any(date.weekday() >= 5 for date in date_range) else 0

# 주말 포함 여부 열 추가 (Yes/No 대신 1과 0 사용)
data['WEEKEND_INCLUDED'] = data.apply(lambda row: 1 if check_weekend_included(row['TRAVEL_START_YMD'], row['TRAVEL_END_YMD']) else 0, axis=1)

# 결과 확인
print(data[['TRAVEL_START_YMD', 'TRAVEL_END_YMD', 'START_DAY_OF_WEEK', 'END_DAY_OF_WEEK', 'WEEKEND_INCLUDED']])


       TRAVEL_START_YMD TRAVEL_END_YMD START_DAY_OF_WEEK END_DAY_OF_WEEK  \
0            2022-08-19     2022-08-22            Friday          Monday   
1            2022-08-19     2022-08-22            Friday          Monday   
2            2022-08-19     2022-08-22            Friday          Monday   
3            2022-08-19     2022-08-22            Friday          Monday   
4            2022-08-19     2022-08-22            Friday          Monday   
...                 ...            ...               ...             ...   
621666              NaT            NaT               NaN             NaN   
621667              NaT            NaT               NaN             NaN   
621668              NaT            NaT               NaN             NaN   
621669              NaT            NaT               NaN             NaN   
621670              NaT            NaT               NaN             NaN   

        WEEKEND_INCLUDED  
0                      1  
1                      1  
2     

2. 계절 변수 추가

In [34]:
# 월별로 계절을 할당하는 함수 정의
def get_season(month):
    if month in [3, 4, 5]:
        return '봄'
    elif month in [6, 7, 8]:
        return '여름'
    elif month in [9, 10, 11]:
        return '가을'
    elif month in [12,1,2]:
        return '겨울'
    else:
        return None

# TRAVEL_START_YMD에서 월 정보를 추출하고, 계절 열 추가
data['SEASON'] = data['TRAVEL_START_YMD'].dt.month.apply(get_season)

# 결과 확인
print(data[['TRAVEL_START_YMD', 'SEASON']])


       TRAVEL_START_YMD SEASON
0            2022-08-19     여름
1            2022-08-19     여름
2            2022-08-19     여름
3            2022-08-19     여름
4            2022-08-19     여름
...                 ...    ...
621666              NaT   None
621667              NaT   None
621668              NaT   None
621669              NaT   None
621670              NaT   None

[621671 rows x 2 columns]


3. 지역이름 통일

In [9]:
data.head()

,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,PAYMENT_NUM,ROAD_NM_CD,LOTNO_CD,PAYMENT_AMT_WON,PAYMENT_ETC,SGG_CD,REL_CD,...,TRAVEL_COMPANIONS_NUM,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_DAY,START_DAY_OF_WEEK,END_DAY_OF_WEEK,WEEKEND_INCLUDED,SEASON
0,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,1.0,...,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,Friday,Monday,1,여름
1,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,1.0,...,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,Friday,Monday,1,여름
2,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,1.0,...,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,Friday,Monday,1,여름
3,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,2.0,...,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,Friday,Monday,1,여름
4,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,2.0,...,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,Friday,Monday,1,여름


4. 총 경비 계산

In [22]:

# 각 TRAVEL_ID에 대해 NaN을 제거하고 중복을 제거하여 합계를 계산한 새로운 TOTAL 컬럼 생성
totals = data.groupby('TRAVEL_ID').agg({
    'PAYMENT_AMT_WON': lambda x: x.dropna().drop_duplicates().sum(),
    'PAYMENT_AMT_WON_lodge': lambda x: x.dropna().drop_duplicates().sum(),
    'PAYMENT_AMT_WON_mvmn': lambda x: x.dropna().drop_duplicates().sum()
}).reset_index()

# 새로운 컬럼명을 TOTAL_AMT, TOTAL_LODGE, TOTAL_MVMN으로 지정
totals.columns = ['TRAVEL_ID', 'TOTAL_AMT', 'TOTAL_LODGE', 'TOTAL_MVMN']

# 기존 데이터프레임에 새로운 TOTAL 컬럼들을 병합
data = data.merge(totals, on='TRAVEL_ID', how='left')

print(data)


        TRAVEL_ID  VISIT_AREA_ID  ACTIVITY_TYPE_CD  PAYMENT_NUM  ROAD_NM_CD  \
0       a_a000007   2.208210e+09               1.0          6.0   3157033.0   
1       a_a000007   2.208210e+09               1.0          6.0   3157033.0   
2       a_a000007   2.208210e+09               1.0          6.0   3157033.0   
3       a_a000007   2.208210e+09               1.0          6.0   3157033.0   
4       a_a000007   2.208210e+09               1.0          6.0   3157033.0   
...           ...            ...               ...          ...         ...   
621666  d_d012413            NaN               NaN          NaN         NaN   
621667  d_d012414            NaN               NaN          NaN         NaN   
621668  d_d012416            NaN               NaN          NaN         NaN   
621669  d_d012442            NaN               NaN          NaN         NaN   
621670  d_d012454            NaN               NaN          NaN         NaN   

            LOTNO_CD  PAYMENT_AMT_WON PAYMENT_ETC  

In [23]:
data.head()

,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,PAYMENT_NUM,ROAD_NM_CD,LOTNO_CD,PAYMENT_AMT_WON,PAYMENT_ETC,SGG_CD,REL_CD,...,TRAVEL_STATUS_ACCOMPANY,TRAVEL_COMPANIONS_NUM,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_DAY,TOTAL_AMT,TOTAL_LODGE,TOTAL_MVMN
0,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,1.0,...,3대 동반 여행(친척 포함),5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,348800.0,1400000.0,294000.0
1,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,1.0,...,3대 동반 여행(친척 포함),5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,348800.0,1400000.0,294000.0
2,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,1.0,...,3대 동반 여행(친척 포함),5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,348800.0,1400000.0,294000.0
3,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,2.0,...,3대 동반 여행(친척 포함),5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,348800.0,1400000.0,294000.0
4,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,2.0,...,3대 동반 여행(친척 포함),5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,348800.0,1400000.0,294000.0


In [29]:
# 특정 TRAVEL_ID에 해당하는 행만 선택하여 payment_amt_won, payment_amt_won_lodge, payment_amt_won_mvmn 값 보기
filtered_data = data[data['TRAVEL_ID'] == 'a_a000007'][['PAYMENT_AMT_WON', 'PAYMENT_AMT_WON_lodge', 'PAYMENT_AMT_WON_mvmn','TOTAL_AMT','TOTAL_LODGE','TOTAL_MVMN']]
print(filtered_data)

     PAYMENT_AMT_WON  PAYMENT_AMT_WON_lodge  PAYMENT_AMT_WON_mvmn  TOTAL_AMT  \
0            12000.0              1400000.0               50000.0   348800.0   
1            12000.0              1400000.0                4000.0   348800.0   
2            12000.0              1400000.0              240000.0   348800.0   
3            12000.0              1400000.0               50000.0   348800.0   
4            12000.0              1400000.0                4000.0   348800.0   
..               ...                    ...                   ...        ...   
130           6000.0              1400000.0                4000.0   348800.0   
131           6000.0              1400000.0              240000.0   348800.0   
132           6000.0              1400000.0               50000.0   348800.0   
133           6000.0              1400000.0                4000.0   348800.0   
134           6000.0              1400000.0              240000.0   348800.0   

     TOTAL_LODGE  TOTAL_MVMN  
0      1

In [31]:
# TOTAL_COST 변수를 추가 (TOTAL_AMT, TOTAL_LODGE, TOTAL_MVMN의 합)
data['TOTAL_COST'] = data['TOTAL_AMT'] + data['TOTAL_LODGE'] + data['TOTAL_MVMN']

data.head()

,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,PAYMENT_NUM,ROAD_NM_CD,LOTNO_CD,PAYMENT_AMT_WON,PAYMENT_ETC,SGG_CD,REL_CD,...,TRAVEL_COMPANIONS_NUM,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_DAY,TOTAL_AMT,TOTAL_LODGE,TOTAL_MVMN,TOTAL_COST
0,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,1.0,...,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,348800.0,1400000.0,294000.0,2042800.0
1,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,1.0,...,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,348800.0,1400000.0,294000.0,2042800.0
2,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,1.0,...,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,348800.0,1400000.0,294000.0,2042800.0
3,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,2.0,...,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,348800.0,1400000.0,294000.0,2042800.0
4,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,2.0,...,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,348800.0,1400000.0,294000.0,2042800.0


5. 필요 변수들만 선택

In [35]:
data.columns

Index(['TRAVEL_ID', 'VISIT_AREA_ID', 'ACTIVITY_TYPE_CD', 'PAYMENT_NUM',
       'ROAD_NM_CD', 'LOTNO_CD', 'PAYMENT_AMT_WON', 'PAYMENT_ETC', 'SGG_CD',
       'REL_CD', 'COMPANION_SITUATION', 'LODGING_TYPE_CD', 'RSVT_YN',
       'PAYMENT_NUM_lodge', 'ROAD_NM_CD_lodge', 'LOTNO_CD_lodge',
       'PAYMENT_AMT_WON_lodge', 'PAYMENT_ETC_lodge', 'MVMN_SE', 'PAYMENT_SE',
       'MVMN_SE_NM', 'PAYMENT_NUM_mvmn', 'PAYMENT_AMT_WON_mvmn',
       'PAYMENT_ETC_mvmn', 'TRAVELER_ID', 'RESIDENCE_SGG_CD', 'GENDER',
       'AGE_GRP', 'INCOME', 'TRAVEL_STATUS_RESIDENCE',
       'TRAVEL_STATUS_DESTINATION', 'TRAVEL_STATUS_ACCOMPANY',
       'TRAVEL_COMPANIONS_NUM', 'TRAVEL_PURPOSE', 'TRAVEL_START_YMD',
       'TRAVEL_END_YMD', 'MVMN_NM', 'TRAVEL_DAY', 'TOTAL_AMT', 'TOTAL_LODGE',
       'TOTAL_MVMN', 'TOTAL_COST', 'START_DAY_OF_WEEK', 'END_DAY_OF_WEEK',
       'WEEKEND_INCLUDED', 'SEASON'],
      dtype='object')

In [38]:
data.head()

,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,PAYMENT_NUM,ROAD_NM_CD,LOTNO_CD,PAYMENT_AMT_WON,PAYMENT_ETC,SGG_CD,REL_CD,...,MVMN_NM,TRAVEL_DAY,TOTAL_AMT,TOTAL_LODGE,TOTAL_MVMN,TOTAL_COST,START_DAY_OF_WEEK,END_DAY_OF_WEEK,WEEKEND_INCLUDED,SEASON
0,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,1.0,...,자가용,4.0,348800.0,1400000.0,294000.0,2042800.0,Friday,Monday,1,여름
1,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,1.0,...,자가용,4.0,348800.0,1400000.0,294000.0,2042800.0,Friday,Monday,1,여름
2,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,1.0,...,자가용,4.0,348800.0,1400000.0,294000.0,2042800.0,Friday,Monday,1,여름
3,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,2.0,...,자가용,4.0,348800.0,1400000.0,294000.0,2042800.0,Friday,Monday,1,여름
4,a_a000007,2.208210e+09,1.0,6.0,3157033.0,2.871033e+09,12000.0,[국수]소치정식,NaN,2.0,...,자가용,4.0,348800.0,1400000.0,294000.0,2042800.0,Friday,Monday,1,여름


In [36]:
# 필요한 열 이름을 리스트로 정의합니다.
selected_columns = [
   'TRAVEL_ID', 'PAYMENT_AMT_WON',
       'PAYMENT_AMT_WON_lodge','PAYMENT_AMT_WON_mvmn',
       'INCOME', 'TRAVEL_STATUS_RESIDENCE',
       'TRAVEL_STATUS_DESTINATION',
       'TRAVEL_COMPANIONS_NUM', 'TRAVEL_PURPOSE', 'TRAVEL_START_YMD',
       'TRAVEL_END_YMD', 'MVMN_NM', 'TRAVEL_DAY', 'TOTAL_COST',
       'WEEKEND_INCLUDED', 'SEASON'
]

# 병합된 데이터프레임(merged_df)에서 필요한 열만 선택합니다.
filtered_df = data[selected_columns]

In [42]:
filtered_df.head()

,TRAVEL_ID,PAYMENT_AMT_WON,PAYMENT_AMT_WON_lodge,PAYMENT_AMT_WON_mvmn,INCOME,TRAVEL_STATUS_RESIDENCE,TRAVEL_STATUS_DESTINATION,TRAVEL_COMPANIONS_NUM,TRAVEL_PURPOSE,TRAVEL_START_YMD,TRAVEL_END_YMD,MVMN_NM,TRAVEL_DAY,TOTAL_COST,WEEKEND_INCLUDED,SEASON
0,a_a000007,12000.0,1400000.0,50000.0,4.0,서울특별시,인천,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,2042800.0,1,여름
1,a_a000007,12000.0,1400000.0,4000.0,4.0,서울특별시,인천,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,2042800.0,1,여름
2,a_a000007,12000.0,1400000.0,240000.0,4.0,서울특별시,인천,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,2042800.0,1,여름
3,a_a000007,12000.0,1400000.0,50000.0,4.0,서울특별시,인천,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,2042800.0,1,여름
4,a_a000007,12000.0,1400000.0,4000.0,4.0,서울특별시,인천,5.0,10;23;24;26;7;,2022-08-19,2022-08-22,자가용,4.0,2042800.0,1,여름


데이터 저장

In [44]:
filtered_df.to_csv('real_data.csv', index=False, encoding='utf-8')

데이터 확인

In [8]:
data['TRAVEL_ID']

0         a_a000007
1         a_a000007
2         a_a000007
3         a_a000007
4         a_a000007
            ...    
621666          NaN
621667          NaN
621668          NaN
621669          NaN
621670          NaN
Name: TRAVEL_ID, Length: 621671, dtype: object

In [7]:
data['TRAVELER_ID']

0         a000007
1         a000007
2         a000007
3         a000007
4         a000007
           ...   
621666    d012413
621667    d012414
621668    d012416
621669    d012442
621670    d012454
Name: TRAVELER_ID, Length: 621671, dtype: object

In [19]:
data['TRAVEL_ID'].isnull().sum()

0

In [18]:
## TRAVEL_ID 결측치 채우기

import pandas as pd

# TRAVEL_ID가 비어 있고, TRAVELER_ID가 존재하는 경우 TRAVELER_ID의 첫 글자를 접두어로 추가하여 TRAVEL_ID를 채우는 코드
data['TRAVEL_ID'] = data.apply(
    lambda row: f"{row['TRAVELER_ID'][0]}_{row['TRAVELER_ID']}" if pd.isna(row['TRAVEL_ID']) and pd.notna(row['TRAVELER_ID']) else row['TRAVEL_ID'], 
    axis=1
)

print(data)


        TRAVEL_ID  VISIT_AREA_ID  ACTIVITY_TYPE_CD  PAYMENT_NUM  ROAD_NM_CD  \
0       a_a000007   2.208210e+09               1.0          6.0   3157033.0   
1       a_a000007   2.208210e+09               1.0          6.0   3157033.0   
2       a_a000007   2.208210e+09               1.0          6.0   3157033.0   
3       a_a000007   2.208210e+09               1.0          6.0   3157033.0   
4       a_a000007   2.208210e+09               1.0          6.0   3157033.0   
...           ...            ...               ...          ...         ...   
621666  d_d012413            NaN               NaN          NaN         NaN   
621667  d_d012414            NaN               NaN          NaN         NaN   
621668  d_d012416            NaN               NaN          NaN         NaN   
621669  d_d012442            NaN               NaN          NaN         NaN   
621670  d_d012454            NaN               NaN          NaN         NaN   

            LOTNO_CD  PAYMENT_AMT_WON PAYMENT_ETC  